<a href="https://colab.research.google.com/github/VyjayantiN/FMML-Projects-Assignments-2023/blob/main/FMML_Module4_Lab2_2024.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# FOUNDATIONS OF MODERN MACHINE LEARNING, IIIT Hyderabad
# Module 4: Perceptron and Gradient Descent
## Lab 2: Introduction to Gradient Descent

Gradient descent is a very important algorithm to understand, as it underpins many of the more advanced algorithms used in Machine Learning and Deep Learning.

A brief overview of the algorithm is


*   start with a random initialization of the solution.
*   incrementally change the solution by moving in the direction of negative gradient of the objective function.
*   repeat the previous step until some convergence criteria is met.

The key equation for change in weight is:
$$w^{k+1} \leftarrow w^k - \eta \Delta J$$



In [ ]:
# Importing the required libraries

import numpy as np
import matplotlib.pyplot as plt
import random

random.seed(42)
np.random.seed(42)

We can start be choosing coefficients for a second degree polynomial equation $(a x^2 + bx + c)$ that will distribute the data we will try to model.

Let's define some random x data (inputs) we hope to predict y (outputs) of.

In [ ]:
def eval_2nd_degree(coeffs, x):
    """
    Function to return the output of evaluating a second degree polynomial,
    given a specific x value.

    Args:
        coeffs: List containing the coefficients a, b, and c for the polynomial.
        x: The input x value to the polynomial.

    Returns:
        y: The corresponding output y value for the second degree polynomial.

    """

    a = coeffs[0] * (x * x)
    b = coeffs[1] * x
    c = coeffs[2]
    y = a + b + c
    return y

hundred_xs = np.random.uniform(-10, 10, 100)
coeffs = [1, 0, 0]

xs = []
ys = []
for x in hundred_xs:
    y  = eval_2nd_degree(coeffs, x)
    xs.append(x)
    ys.append(y)

In [ ]:
plt.plot(xs, ys, 'g+')
plt.title('Original data')
plt.show()

This is good, but we could improve on this by making things more realistic. You can add noise or **jitter** to the values so they can resemble real-world data.


In [ ]:
def eval_2nd_degree_jitter(coeffs, x, j):
    """
    Function to return the noisy output of evaluating a second degree polynomial,
    given a specific x value. Output values can be within [y − j, y + j].

    Args:
        coeffs: List containing the coefficients a, b, and c for the polynomial.
        x: The input x value to the polynomial.
        j: Jitter parameter, to introduce noise to output y.

    Returns:
        y: The corresponding jittered output y value for the second degree polynomial.

    """

    a = coeffs[0] * (x * x)
    b = coeffs[1] * x
    c = coeffs[2]
    y = a + b + c

    interval = [y - j, y + j]
    interval_min = interval[0]
    interval_max = interval[1]
    jit_val = random.random() * interval_max      # Generate a random number in range 0 to interval max

    while interval_min > jit_val:                 # While the random jitter value is less than the interval min,
        jit_val = random.random() * interval_max  # it is not in the right range. Re-roll the generator until it
                                                  # give a number greater than the interval min.

    return jit_val

xs = []
ys = []
for x in hundred_xs:
    y  = eval_2nd_degree_jitter(coeffs, x, 0.1)
    xs.append(x)
    ys.append(y)


In [ ]:
plt.figure(figsize=(12, 6))
plt.plot(xs, ys, 'g+')
plt.title('Original data with jitter')
plt.show()

We will now build our predictive model, and optimize it with gradient descent and we will try to get as close to these values as possible.

To get a quantifiable measure of how incorrect it is, we calculate the Mean Squared Error loss for the model. This is the mean value of the sum of the squared differences between the actual and predicted outputs.

$$ E = \frac{1}{n} \sum_{i=0}^n (y_i - \bar{y_i})^2 $$

In [ ]:
def loss_mse(ys, y_bar):
    """
    Calculates MSE loss.

    Args:
        ys: training data labels
        y_bar: prediction labels

    Returns: Calculated MSE loss.
    """

    return sum((ys - y_bar) * (ys - y_bar)) / len(ys)

In [ ]:
rand_coeffs = (random.randrange(-10, 10), random.randrange(-10, 10), random.randrange(-10, 10))
y_bar = eval_2nd_degree(rand_coeffs, hundred_xs)
plt.figure(figsize=(12, 6))
plt.plot(xs, ys, 'g+', label = 'original')
plt.plot(xs, y_bar, 'ro', label = 'prediction')
plt.title('Original data vs first prediction')
plt.legend(loc="lower right")
plt.show()

In [ ]:
initial_model_loss = loss_mse(ys, y_bar)
initial_model_loss

We can see that the loss is quite a large number. Let’s now see if we can improve on this fairly high loss metric by optimizing the model with gradient descent.

We wish to improve our model. Therefore we want to alter its coefficients $a$, $b$ and $c$ to decrease the error. Therefore we require knowledge about how each coefficient affects the error. This is achieved by calculating the partial derivative of the loss function with respect to **each** of the individual coefficients.

In [ ]:
def calc_gradient_2nd_poly(rand_coeffs, hundred_xs, ys):
    """
    calculates the gradient for a second degree polynomial.

    Args:
        coeffs: a,b and c, for a 2nd degree polynomial [ y = ax^2 + bx + c ]
        inputs_x: x input datapoints
        outputs_y: actual y output points

    Returns: Calculated gradients for the 2nd degree polynomial, as a tuple of its parts for a,b,c respectively.

    """

    a_s = []
    b_s = []
    c_s = []

    y_bars = eval_2nd_degree(rand_coeffs, hundred_xs)

    for x, y, y_bar in list(zip(hundred_xs, ys, y_bars)):    # take tuple of (x datapoint, actual y label, predicted y label)
        x_squared = x ** 2
        partial_a = x_squared * (y - y_bar)
        a_s.append(partial_a)
        partial_b = x * (y - y_bar)
        b_s.append(partial_b)
        partial_c = (y - y_bar)
        c_s.append(partial_c)

    num = [i for i in y_bars]
    n = len(num)

    gradient_a = (-2 / n) * sum(a_s)
    gradient_b = (-2 / n) * sum(b_s)
    gradient_c = (-2 / n) * sum(c_s)

    return (gradient_a, gradient_b, gradient_c)   # return calculated gradients as a a tuple of its 3 parts

In [ ]:
calc_grad = calc_gradient_2nd_poly(rand_coeffs, hundred_xs, ys)

lr = 0.0001
a_new = rand_coeffs[0] - lr * calc_grad[0]
b_new = rand_coeffs[1] - lr * calc_grad[1]
c_new = rand_coeffs[2] - lr * calc_grad[2]

new_model_coeffs = (a_new, b_new, c_new)
print(f"New model coeffs: {new_model_coeffs}")

# update with these new coeffs:
new_y_bar = eval_2nd_degree(new_model_coeffs, hundred_xs)
updated_model_loss = loss_mse(ys, new_y_bar)

print(f"Now have smaller model loss: {updated_model_loss} vs {initial_model_loss}")

In [ ]:
plt.figure(figsize=(12, 6))
plt.plot(xs, ys, 'g+', label = 'original model')
plt.plot(xs, y_bar, 'ro', label = 'first prediction')
plt.plot(xs, new_y_bar, 'b.', label = 'updated prediction')
plt.title('Original model vs 1st prediction vs updated prediction with lower loss')
plt.legend(loc="lower right")
plt.show()

We’re almost ready. The last step will be to perform gradient descent iteratively over a number of epochs (cycles or iterations.) With every epoch we hope to see an improvement in the form of lowered loss, and better model-fitting to the original data.

In [ ]:
def calc_gradient_2nd_poly_for_GD(coeffs, inputs_x, outputs_y, lr):
    """
    calculates the gradient for a second degree polynomial.

    Args:
        coeffs: a,b and c, for a 2nd degree polynomial [ y = ax^2 + bx + c ]
        inputs_x: x input datapoints
        outputs_y: actual y output points
        lr: learning rate

    Returns: Calculated gradients for the 2nd degree polynomial, as a tuple of its parts for a,b,c respectively.

    """
    a_s = []
    b_s = []
    c_s = []

    y_bars = eval_2nd_degree(coeffs, inputs_x)

    for x,y,y_bar in list(zip(inputs_x, outputs_y, y_bars)):    # take tuple of (x datapoint, actual y label, predicted y label)
        x_squared = x ** 2
        partial_a = x_squared * (y - y_bar)
        a_s.append(partial_a)
        partial_b = x * (y - y_bar)
        b_s.append(partial_b)
        partial_c = (y - y_bar)
        c_s.append(partial_c)

    num = [i for i in y_bars]
    n = len(num)

    gradient_a = (-2 / n) * sum(a_s)
    gradient_b = (-2 / n) * sum(b_s)
    gradient_c = (-2 / n) * sum(c_s)


    a_new = coeffs[0] - lr * gradient_a
    b_new = coeffs[1] - lr * gradient_b
    c_new = coeffs[2] - lr * gradient_c

    new_model_coeffs = (a_new, b_new, c_new)

    # update with these new coeffs:
    new_y_bar = eval_2nd_degree(new_model_coeffs, inputs_x)

    updated_model_loss = loss_mse(outputs_y, new_y_bar)
    return updated_model_loss, new_model_coeffs, new_y_bar

In [ ]:
def gradient_descent(epochs, lr):
    """
    Perform gradient descent for a second degree polynomial.

    Args:
        epochs: number of iterations to perform of finding new coefficients and updatingt loss.
        lr: specified learning rate

    Returns: Tuple containing (updated_model_loss, new_model_coeffs, new_y_bar predictions, saved loss updates)

    """
    losses = []
    rand_coeffs_to_test = rand_coeffs
    for i in range(epochs):
        loss = calc_gradient_2nd_poly_for_GD(rand_coeffs_to_test, hundred_xs, ys, lr)
        rand_coeffs_to_test = loss[1]
        losses.append(loss[0])
    print(losses)
    return loss[0], loss[1], loss[2], losses  # (updated_model_loss, new_model_coeffs, new_y_bar, saved loss updates)

In [ ]:
GD = gradient_descent(30000, 0.0003)


In [ ]:
plt.figure(figsize=(12,6))
plt.plot(xs, ys, 'g+', label = 'original')
plt.plot(xs, GD[2], 'b.', label = 'final_prediction')
plt.title('Original vs Final prediction after Gradient Descent')
plt.legend(loc = "lower right")
plt.show()

This trained model is showing vast improvements after it’s full training cycle. We can examine further by inspecting its final predicted coefficients $a$, $b$ and $c$.

In [ ]:
print(f"Final Coefficients predicted: {GD[1]}")
print(f"Original Coefficients: {coeffs}")

Not too far off! A big improvement over the initial random model. Looking at the plot of the loss reduction over training offers further insights.


In [ ]:
plt.figure(figsize=(12, 6))
plt.plot(GD[3], 'b-', label = 'loss')
# plt.xlim(0,50)
plt.title('Loss over 500 iterations')
plt.legend(loc="lower right")
plt.xlim((0,100))
plt.xlabel('Iterations')
plt.ylabel('MSE')
plt.show()

We observe that the model loss reached close to zero, to give us our more accurate coefficients. We can also see there was no major improvement in loss after about 100 epochs. An alternative strategy would be to add some kind of condition to the training step that stops training when a certain minimum loss threshold has been reached. This would prevent excessive training and potential over-fitting for the model.

# Things to try



1.   Change the coefficients array and try a different polynomial instead of our $x^2$.
2.   Increase/decrease the learning rate to see how many iterations will be take to coverge. Does it even converge on a huge learning rate?
3. Take a degree 5 polynomial with 5 roots and try different initializations, instead of random ones. Does it converge on different values for different initializations? Why does initialization not matter in our case of $x^2$?
4. Can you modify the algorithm to find a maxima of a function, instead of a minima?



# **ANSWERS**

# **QUESTION 1**

In [ ]:
#  QUESTION 1
import numpy as np
import random
import matplotlib.pyplot as plt

# Original code for polynomial evaluation
def eval_2nd_degree(coeffs, x):
    a = coeffs[0] * (x * x)
    b = coeffs[1] * x
    c = coeffs[2]
    y = a + b + c
    return y

# Generate random x values
hundred_xs = np.random.uniform(-10, 10, 100)
# Original coefficients for the polynomial
original_coeffs = [1, 0, 0]
# Evaluate the original polynomial
ys_original = [eval_2nd_degree(original_coeffs, x) for x in hundred_xs]

# New coefficients for a different polynomial
new_coeffs = [2, -3, 1]
# Evaluate the new polynomial
ys_new = [eval_2nd_degree(new_coeffs, x) for x in hundred_xs]

# Plot the original and new polynomials with different colors
plt.plot(hundred_xs, ys_original, 'g+', label='Original Polynomial')
plt.plot(hundred_xs, ys_new, 'b+', label='New Polynomial')
plt.title('Original and New Polynomials')
plt.legend()
plt.show()


# **QUESTION 2**

The learning rate is a crucial hyperparameter in gradient descent. It determines the size of the steps taken during each iteration. Setting it too high might cause overshooting and prevent convergence, while setting it too low might result in slow convergence.

In [ ]:
# QUESTION 2
import numpy as np
import random
import matplotlib.pyplot as plt

# Original code for gradient descent
# ...

# Function to perform gradient descent with a specified learning rate
def gradient_descent_lr(epochs, lr):
    losses = []
    rand_coeffs_to_test = rand_coeffs
    for i in range(epochs):
        loss = calc_gradient_2nd_poly_for_GD(rand_coeffs_to_test, hundred_xs, ys, lr)
        rand_coeffs_to_test = loss[1]
        losses.append(loss[0])

    # Print the losses to observe convergence
    print(f"Losses with learning rate {lr}: {losses}")

    return loss[0], loss[1], loss[2], losses

# Try different learning rates
high_lr = 0.01
low_lr = 0.00001
GD_high_lr = gradient_descent_lr(30000, high_lr)
GD_low_lr = gradient_descent_lr(30000, low_lr)

# Plot the loss curves for different learning rates
plt.figure(figsize=(12, 6))
plt.plot(GD_high_lr[3], label=f'High Learning Rate ({high_lr})')
plt.plot(GD_low_lr[3], label=f'Low Learning Rate ({low_lr})')
plt.title('Loss over Iterations with Different Learning Rates')
plt.xlabel('Iterations')
plt.ylabel('MSE Loss')
plt.legend()
plt.show()


# **QUESTION 3**

Yes, it is likely that the gradient descent will converge to different values for the coefficients with different initializations.
The convergence behavior depends on the shape of the optimization landscape, and different initializations can lead the model to different local minima.

For the case of a degree 2 polynomial, such as
ax^2+bx+c,  the optimization landscape is simpler and generally has a single global minimum.
The quadratic function is convex, meaning it has a well-defined minimum point, and the optimization process will naturally converge to that minimum, regardless of the initial values.


In [ ]:
# QUESTION 3
import numpy as np
import random
import matplotlib.pyplot as plt
import warnings

# Suppress runtime warnings
warnings.filterwarnings("ignore", category=RuntimeWarning)

# Original code for gradient descent
def eval_5th_degree(coeffs, x):
    a = coeffs[0] * (x ** 5)
    b = coeffs[1] * (x ** 4)
    c = coeffs[2] * (x ** 3)
    d = coeffs[3] * (x ** 2)
    e = coeffs[4] * x
    f = coeffs[5]
    y = a + b + c + d + e + f
    return y

def calc_gradient_5th_poly(coeffs, inputs_x, outputs_y):
    a_s = []
    b_s = []
    c_s = []
    d_s = []
    e_s = []
    f_s = []

    y_bars = eval_5th_degree(coeffs, inputs_x)

    for x, y, y_bar in list(zip(inputs_x, outputs_y, y_bars)):
        x_pow_5 = x ** 5
        x_pow_4 = x ** 4
        x_pow_3 = x ** 3
        x_pow_2 = x ** 2
        x_pow_1 = x
        partial_a = x_pow_5 * (y - y_bar)
        a_s.append(partial_a)
        partial_b = x_pow_4 * (y - y_bar)
        b_s.append(partial_b)
        partial_c = x_pow_3 * (y - y_bar)
        c_s.append(partial_c)
        partial_d = x_pow_2 * (y - y_bar)
        d_s.append(partial_d)
        partial_e = x_pow_1 * (y - y_bar)
        e_s.append(partial_e)
        partial_f = (y - y_bar)
        f_s.append(partial_f)

    n = len(outputs_y)

    gradient_a = (-2 / n) * sum(a_s)
    gradient_b = (-2 / n) * sum(b_s)
    gradient_c = (-2 / n) * sum(c_s)
    gradient_d = (-2 / n) * sum(d_s)
    gradient_e = (-2 / n) * sum(e_s)
    gradient_f = (-2 / n) * sum(f_s)

    return (gradient_a, gradient_b, gradient_c, gradient_d, gradient_e, gradient_f)

def gradient_descent_5th_degree(epochs, lr, coeffs):
    losses = []
    rand_coeffs_to_test = coeffs
    for i in range(epochs):
        loss = calc_gradient_5th_poly(rand_coeffs_to_test, hundred_xs, ys)
        rand_coeffs_to_test = tuple(c - lr * g for c, g in zip(rand_coeffs_to_test, loss))
        losses.append(loss[0])

    print(f"Losses for coefficients {coeffs}: {losses}")

    return loss[0], rand_coeffs_to_test, [eval_5th_degree(rand_coeffs_to_test, x) for x in hundred_xs]

# Generate random x values
hundred_xs = np.random.uniform(-1, 1, 100)  # Normalizing the x values

# Coefficients for a degree 5 polynomial with 5 roots
degree_5_coeffs = [1, -4, 4, -4, 1, 0]

# Random initialization for coefficients
rand_coeffs_5_init_1 = tuple(random.uniform(-1, 1) for _ in range(6))  # Normalizing the initial coefficients
rand_coeffs_5_init_2 = tuple(random.uniform(-1, 1) for _ in range(6))

# Perform gradient descent with different initializations
GD_degree_5_rand_init_1 = gradient_descent_5th_degree(30000, 0.0001, rand_coeffs_5_init_1)
GD_degree_5_rand_init_2 = gradient_descent_5th_degree(30000, 0.0001, rand_coeffs_5_init_2)
GD_degree_5_fixed_init = gradient_descent_5th_degree(30000, 0.0001, degree_5_coeffs)

# Plot the results for different initializations
plt.figure(figsize=(12, 6))
plt.plot(hundred_xs, GD_degree_5_rand_init_1[2], 'b.', label='Random Initialization 1')
plt.plot(hundred_xs, GD_degree_5_rand_init_2[2], 'r.', label='Random Initialization 2')
plt.plot(hundred_xs, GD_degree_5_fixed_init[2], 'g.', label='Fixed Initialization')
plt.title('Degree 5 Polynomial - Convergence with Different Initializations')
plt.legend()
plt.show()



# **QUESTION 4**

In [ ]:
# QUESTION 4
import numpy as np
import random
import matplotlib.pyplot as plt
import warnings

# Suppress runtime warnings
warnings.filterwarnings("ignore", category=RuntimeWarning)

# Original code for gradient ascent (maximization)
def eval_5th_degree(coeffs, x):
    a, b, c, d, e, f = coeffs
    y = a * (x ** 5) + b * (x ** 4) + c * (x ** 3) + d * (x ** 2) + e * x + f
    return y

def calc_gradient_5th_poly(coeffs, inputs_x, outputs_y):
    a, b, c, d, e, f = coeffs
    a_s, b_s, c_s, d_s, e_s, f_s = 0, 0, 0, 0, 0, 0

    y_bars = eval_5th_degree(coeffs, inputs_x)

    for x, y, y_bar in list(zip(inputs_x, outputs_y, y_bars)):
        x_pow_5 = x ** 5
        x_pow_4 = x ** 4
        x_pow_3 = x ** 3
        x_pow_2 = x ** 2
        x_pow_1 = x

        a_s += x_pow_5 * (y_bar - y)
        b_s += x_pow_4 * (y_bar - y)
        c_s += x_pow_3 * (y_bar - y)
        d_s += x_pow_2 * (y_bar - y)
        e_s += x_pow_1 * (y_bar - y)
        f_s += (y_bar - y)

    n = len(outputs_y)

    gradient_a = (2 / n) * a_s
    gradient_b = (2 / n) * b_s
    gradient_c = (2 / n) * c_s
    gradient_d = (2 / n) * d_s
    gradient_e = (2 / n) * e_s
    gradient_f = (2 / n) * f_s

    return (gradient_a, gradient_b, gradient_c, gradient_d, gradient_e, gradient_f)

def gradient_ascent_5th_degree(epochs, lr, coeffs):
    losses = []
    rand_coeffs_to_test = coeffs
    for i in range(epochs):
        loss = calc_gradient_5th_poly(rand_coeffs_to_test, hundred_xs, ys)
        rand_coeffs_to_test = tuple(c + lr * g for c, g in zip(rand_coeffs_to_test, loss))
        losses.append(loss[0])

    print(f"Losses for coefficients {coeffs}: {losses}")

    return loss[0], rand_coeffs_to_test, [eval_5th_degree(rand_coeffs_to_test, x) for x in hundred_xs]

# Generate random x values
hundred_xs = np.random.uniform(-1, 1, 100)  # Normalizing the x values

# Coefficients for a degree 5 polynomial with 5 roots
degree_5_coeffs = [1, -4, 4, -4, 1, 0]

# Random initialization for coefficients
rand_coeffs_5_init_1 = tuple(random.uniform(-1, 1) for _ in range(6))  # Normalizing the initial coefficients
rand_coeffs_5_init_2 = tuple(random.uniform(-1, 1) for _ in range(6))

# Perform gradient ascent with different initializations
GA_degree_5_rand_init_1 = gradient_ascent_5th_degree(30000, 0.0001, rand_coeffs_5_init_1)
GA_degree_5_rand_init_2 = gradient_ascent_5th_degree(30000, 0.0001, rand_coeffs_5_init_2)
GA_degree_5_fixed_init = gradient_ascent_5th_degree(30000, 0.0001, degree_5_coeffs)

# Plot the results for different initializations
plt.figure(figsize=(12, 6))
plt.plot(hundred_xs, GA_degree_5_rand_init_1[2], 'b.', label='Random Initialization 1')
plt.plot(hundred_xs, GA_degree_5_rand_init_2[2], 'r.', label='Random Initialization 2')
plt.plot(hundred_xs, GA_degree_5_fixed_init[2], 'g.', label='Fixed Initialization')
plt.title('Degree 5 Polynomial - Convergence with Different Initializations (Maximization)')
plt.legend()
plt.show()
